In [1]:
!git clone https://github.com/emanhamed/Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 39.50 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Updating files: 100% (2144/2144), done.


In [35]:
import os
import locale
import glob
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras import layers
from keras.optimizers import Adam
from keras.activations import relu, linear
from keras.losses import mean_absolute_percentage_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [33]:
DATASET_PATH: str = '/content/Houses-dataset/Houses Dataset'
MODEL_PATH_TO_SAVE: str = '/content/drive/MyDrive/weights/house_pricing_best.keras'
TEST_IMAGES_DIRECTORY: str = '/content/test_images'

## Utils

In [38]:
def load_house_images(df: pd.DataFrame, input_path: str) -> np.ndarray:
    images = []

    for i in tqdm(df.index.values):

        base_path = os.path.sep.join([input_path, f'{i + 1}_*'])
        house_paths = sorted(list(
            glob.glob(base_path)
        ))

        input_images = []
        output_images = np.zeros((64, 64, 3), dtype='uint8')

        for house_path in house_paths:
            image = cv.imread(house_path)
            image = cv.resize(image, (32, 32))
            image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
            input_images.append(image)

        output_images[0:32, 0:32] = input_images[0]
        output_images[0:32, 32:64] = input_images[1]
        output_images[32:64, 32:64] = input_images[2]
        output_images[32:64, 0:32] = input_images[3]

        images.append(output_images)

    return np.array(images)

In [39]:
def load_house_attributes(input_path: str) -> pd.DataFrame:
    columns = ['bedrooms', 'bathrooms', 'area', 'zipcode', 'price']
    df = pd.read_csv(input_path, sep=" ", header=None, names=columns)

    zipcodes = df['zipcode'].value_counts().keys().tolist()
    counts = df['zipcode'].value_counts().tolist()

    for zipcode, count in tqdm(zip(zipcodes, counts)):
        if count < 25:
            index = df[df['zipcode'] == zipcode].index
            df.drop(index, inplace=True)

    return df

In [40]:
def create_cnn(width: int, height: int, depth: int, filters: tuple[int, int, int] = (16, 32, 64), regress: bool = False) -> Sequential:
    input_shape = (height, width, depth)
    chan_dim = -1

    model: Sequential = Sequential()
    for i, f in tqdm(enumerate(filters)):

        if i == 0:
            model.add(layers.Conv2D(f, (3, 3), padding='same',
                      input_shape=input_shape, activation=relu))

        else:
            model.add(layers.Conv2D(f, (3, 3), padding='same', activation=relu))

        model.add(layers.BatchNormalization(axis=chan_dim))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(16, activation=relu))
    model.add(layers.BatchNormalization(axis=chan_dim))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation=relu))

    if regress:
        model.add(layers.Dense(1, activation=linear))

    return model


## Training

In [41]:
print('[INFO] loading house attributes...')
input_path = os.path.sep.join([DATASET_PATH, 'HousesInfo.txt'])
df = load_house_attributes(input_path)
print('\n[INFO] loading house attributes done.')

[INFO] loading house attributes...


49it [00:00, 1228.00it/s]


[INFO] loading house attributes done.


In [42]:
print('[INFO] loading house images...')
images = load_house_images(df, DATASET_PATH)
images = images / 255.
train_attr_x, test_attr_x, train_images_x, test_images_x = train_test_split(
    df, images, test_size=.2, random_state=42
)
print('\n[INFO] loading house images done')

[INFO] loading house images...


100%|██████████| 362/362 [00:10<00:00, 35.46it/s]


[INFO] loading house images done


In [43]:
max_price = train_attr_x['price'].max()
y_train = train_attr_x['price'] / max_price
y_test = test_attr_x['price'] / max_price

In [44]:
print('[INFO] creating the model...')
model = create_cnn(64, 64, 3, regress=True)
model.compile(optimizer=Adam(learning_rate=1e-3), #, decay=1e-3 / 200),
              loss=mean_absolute_percentage_error)
checkpoint = ModelCheckpoint(MODEL_PATH_TO_SAVE, save_best_only=True)
print('\n[INFO] creating the model done')

[INFO] creating the model...


3it [00:00, 33.03it/s]


[INFO] creating the model done


In [45]:
print('[INFO] training model...')
model.fit(
    x=train_images_x,
    y=y_train,
    validation_data=(test_images_x, y_test),
    epochs=200,
    batch_size=8,
    callbacks=[checkpoint]
)
print('[INFO] training model done')

[INFO] training model...
Epoch 1/200
37/37 [==============================] - 4s 20ms/step - loss: 1503.7213 - val_loss: 1320.0948
Epoch 2/200
37/37 [==============================] - 0s 11ms/step - loss: 1021.3154 - val_loss: 3942.3376
Epoch 3/200
37/37 [==============================] - 0s 11ms/step - loss: 1018.7276 - val_loss: 4610.8052
Epoch 4/200
37/37 [==============================] - 0s 11ms/step - loss: 708.1383 - val_loss: 4105.4624
Epoch 5/200
37/37 [==============================] - 0s 7ms/step - loss: 590.7356 - val_loss: 2533.7319
Epoch 6/200
37/37 [==============================] - 0s 7ms/step - loss: 386.6475 - val_loss: 1705.5995
Epoch 7/200
37/37 [==============================] - 0s 8ms/step - loss: 322.0567 - val_loss: 1483.2529
Epoch 8/200
37/37 [==============================] - 0s 10ms/step - loss: 278.5504 - val_loss: 1134.1636
Epoch 9/200
37/37 [==============================] - 0s 10ms/step - loss: 275.4168 - val_loss: 703.1582
Epoch 10/200
37/37 [===========

In [47]:
model: Sequential = load_model(MODEL_PATH_TO_SAVE)
model.evaluate(test_images_x, y_test)

3/3 [==============================] - 0s 6ms/step - loss: 50.5028


50.5028076171875

## Inference

In [51]:
def predict(model_save_path: str, input_images_path: str) -> float:
    # del model
    model: Sequential = load_model(model_save_path)

    images = []
    output_image = np.zeros((64, 64, 3), dtype='uint8')

    for img in os.listdir(input_images_path):
        image = cv.imread(os.path.join(input_images_path, img))
        image = cv.resize(image, (32, 32))
        images.append(image)

    output_image[0:32, 0:32] = images[0]
    output_image[0:32, 32:64] = images[1]
    output_image[32:64, 32:64] = images[2]
    output_image[32:64, 0:32] = images[3]
    output_image = cv.cvtColor(output_image, cv.COLOR_BGR2RGB)
    plt.imshow(output_image)

    output_image = np.array(output_image)
    output_image = output_image / 255.

    output_image = output_image.reshape(1, 64, 64, 3)
    prediction = model.predict([output_image])
    return prediction

In [ ]:
prediction: float = predict(MODEL_PATH_TO_SAVE, TEST_IMAGES_DIRECTORY)
print(f'House price estimated: {prediction[0][0]}')